In [1]:
import sqlite3

In [2]:
from sqlalchemy import create_engine 

In [17]:
import pandas as pd

In [3]:
database = "/Users/natakat/sqlite_db_pythonsqlite.db"

In [4]:
conn = sqlite3.connect(database)

In [5]:
print(sqlite3.version)

2.6.0


In [6]:
cur = conn.cursor()

In [7]:
query1 = """
        SELECT *
        FROM Facilities
        """

In [8]:
cur.execute(query1)

In [10]:
rows = cur.fetchall()

In [11]:
print(rows)

[(0, 'Tennis Court 1', 5, 25, 10000, 200), (1, 'Tennis Court 2', 5, 25, 8000, 200), (2, 'Badminton Court', 0, 15.5, 4000, 50), (3, 'Table Tennis', 0, 5, 320, 10), (4, 'Massage Room 1', 9.9, 80, 4000, 3000), (5, 'Massage Room 2', 9.9, 80, 4000, 3000), (6, 'Squash Court', 3.5, 17.5, 5000, 80), (7, 'Snooker Table', 0, 5, 450, 15), (8, 'Pool Table', 0, 5, 400, 15)]


In [18]:
df = pd.read_sql_query("SELECT * FROM Facilities", conn)


In [19]:
print(df)

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   
5      5   Massage Room 2         9.9       80.0           4000   
6      6     Squash Court         3.5       17.5           5000   
7      7    Snooker Table         0.0        5.0            450   
8      8       Pool Table         0.0        5.0            400   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  
5                3000  
6                  80  
7                  15  
8                  15  


In [24]:
qm = "SELECT f.name AS facility, SUM(b.slots * f.membercost) as rev FROM Bookings AS b, Facilities AS f, Members AS m WHERE b.facid = f.facid AND b.memid = m.memid AND m.memid > 0 GROUP BY facility"

In [25]:
df_m = pd.read_sql_query(qm, conn)
df_m

,facility,rev
0,Badminton Court,0.0
1,Massage Room 1,8751.6
2,Massage Room 2,534.6
3,Pool Table,0.0
4,Snooker Table,0.0
5,Squash Court,1463.0
6,Table Tennis,0.0
7,Tennis Court 1,4785.0
8,Tennis Court 2,4410.0


In [26]:
qg = "SELECT f.name AS facility, SUM(b.slots * f.guestcost) AS rev FROM Bookings AS b, Facilities AS f, Members AS m WHERE b.facid = f.facid AND b.memid = m.memid AND m.memid = 0 GROUP BY Facility"

In [27]:
df_g = pd.read_sql_query(qg, conn)
df_g

,facility,rev
0,Badminton Court,1906.5
1,Massage Room 1,41600.0
2,Massage Room 2,13920.0
3,Pool Table,270.0
4,Snooker Table,240.0
5,Squash Court,12005.0
6,Table Tennis,180.0
7,Tennis Court 1,9075.0
8,Tennis Court 2,9900.0


In [28]:
# total revenue < 1000
total_rev_q = "SELECT memr.mfacility as facility, (memr.rev + guestr.rev) as total_revenue FROM (SELECT f.name AS mfacility, SUM(b.slots * f.membercost) as rev FROM Bookings AS b, Facilities AS f, Members AS m WHERE b.facid = f.facid AND b.memid = m.memid AND m.memid > 0 GROUP BY mfacility) as memr LEFT JOIN (SELECT f.name AS gfacility, SUM(b.slots * f.guestcost) AS rev FROM Bookings AS b, Facilities AS f, Members AS m WHERE b.facid = f.facid AND b.memid = m.memid AND m.memid = 0 GROUP BY gfacility) as guestr ON memr.mfacility = guestr.gfacility WHERE (memr.rev + guestr.rev) < 1000 ORDER BY facility"


In [29]:
df_total = pd.read_sql_query(total_rev_q, conn)
df_total

,facility,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


In [56]:
rec_mem_q = "SELECT m.surname, m.firstname, r.surname as recommended_by_lastname, r.firstname as recommended_by_firstname FROM Members AS m LEFT JOIN Members as r ON m.recommendedby = r.memid WHERE m.memid != 0 ORDER BY m.surname"


In [57]:
df_rec_mem_q = pd.read_sql_query(rec_mem_q, conn)
df_rec_mem_q

,surname,firstname,recommended_by_lastname,recommended_by_firstname
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren
5,Coplin,Joan,Baker,Timothy
6,Crumpet,Erica,Smith,Tracy
7,Dare,Nancy,Joplette,Janice
8,Farrell,Jemima,None,None
9,Farrell,David,None,None


In [32]:
q12 = "SELECT f.name AS facility, count(b.slots) AS used_by_members FROM Bookings as b, Facilities as f WHERE b.facid = f.facid AND memid > 0 GROUP BY facility"


In [33]:
df_12 = pd.read_sql_query(q12, conn)
df_12

,facility,used_by_members
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


In [54]:
q_by_month = "SELECT f.name AS facility, count(b.slots) AS used_by_members, strftime('%m',b.starttime) as month, strftime('%Y',b.starttime) as year FROM Bookings as b, Facilities as f WHERE b.facid = f.facid AND memid > 0 GROUP BY facility, year"


In [55]:
df_by_month = pd.read_sql_query(q_by_month, conn)
df_by_month

,facility,used_by_members,month,year
0,Badminton Court,344,07,2012
1,Massage Room 1,421,07,2012
2,Massage Room 2,27,07,2012
3,Pool Table,783,07,2012
4,Snooker Table,421,07,2012
5,Squash Court,195,07,2012
6,Table Tennis,385,07,2012
7,Tennis Court 1,308,07,2012
8,Tennis Court 2,276,07,2012


In [ ]:
conn.close()